In [0]:
!pip install tensorflow==2.1.0
!pip install tokenizers
!pip install git+https://github.com/huggingface/transformers

     |████████████████████████████████| 421.8MB 42kB/s 
     |████████████████████████████████| 450kB 37.7MB/s 
     |████████████████████████████████| 3.9MB 42.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=55c4afcb25702b25038bf401cd84fcf50a6deddfcff18d77de66995e3a56ea72
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installatio

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
from tensorflow.keras import backend as K

from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers, losses, models, callbacks
from tensorflow.keras.initializers import *
from tensorflow.keras.regularizers import *
from tensorflow.keras.constraints import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *

import warnings
warnings.filterwarnings('ignore')

import math
import glob
import sys
import gc
import os
import re
import random

import transformers
from transformers import *
import tokenizers

print("Tensorflow version: ", tf.__version__)
print("Transformers version: ",transformers.__version__)
print("Tokenizers version: ", tokenizers.__version__)
print("Tensorflow GPU available: ",tf.test.is_gpu_available)
print("Pytorch GPU available: ", torch.cuda.is_available())

Tensorflow version:  2.1.0
Transformers version:  2.8.0
Tokenizers version:  0.7.0
Tensorflow GPU available:  <function is_gpu_available at 0x7f446109aae8>
Pytorch GPU available:  True


In [0]:
%%time
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer
from glob import glob
paths = list(
    glob("/content/drive/My Drive/pretrain_data/*.txt")
)
# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer(lowercase=False)

# Customize training
tokenizer.train(files=paths, vocab_size=50265, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# Save files to disk
import os
OUT_DIR = "TweetBERTa"
os.makedirs(OUT_DIR, exist_ok=True)
tokenizer.save(OUT_DIR)

CPU times: user 48.3 s, sys: 28.9 s, total: 1min 17s
Wall time: 22.7 s


In [0]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./TweetBERTa/vocab.json",
    "./TweetBERTa/merges.txt",
)

In [0]:
import json
config = {
	"architectures": [
		"RobertaForMaskedLM"
	],
	"attention_probs_dropout_prob": 0.1,
	"hidden_act": "gelu",
	"hidden_dropout_prob": 0.1,
	"hidden_size": 768,
	"initializer_range": 0.02,
	"intermediate_size": 3072,
	"layer_norm_eps": 1e-05,
	"max_position_embeddings": 514,
	"model_type": "roberta",
	"num_attention_heads": 12,
	"num_hidden_layers": 6,
	"type_vocab_size": 1,
	"vocab_size": 50265
}

with open("./TweetBERTa/config.json", 'w') as fp:
    json.dump(config, fp)

tokenizer_config = {
	"max_len": 512
}
with open("./TweetBERTa/tokenizer_config.json", 'w') as fp:
    json.dump(tokenizer_config, fp)

In [0]:
#https://zablo.net/blog/post/training-roberta-from-scratch-the-missing-guide-polish-language-model/?utm_source=blog&utm_medium=link&utm_campaign=roberta_from_scratch
!wget -c https://raw.githubusercontent.com/arctic-yen/Pretraining-RoBERTa/master/run_language_modeling_with_tokenizers.py

!wget -c https://raw.githubusercontent.com/huggingface/transformers/master/examples/run_language_modeling.py

--2020-05-06 03:11:16--  https://raw.githubusercontent.com/arctic-yen/Pretraining-RoBERTa/master/run_language_modeling_with_tokenizers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36520 (36K) [text/plain]
Saving to: ‘run_language_modeling_with_tokenizers.py’

run_language_modeli 100%[===================>]  35.66K  --.-KB/s    in 0.01s   

2020-05-06 03:11:16 (3.06 MB/s) - ‘run_language_modeling_with_tokenizers.py’ saved [36520/36520]

--2020-05-06 03:11:18--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/run_language_modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:4

In [0]:
# run_language_modeling_with_tokenizers.py -- it's the version with support for fast tokenizers, see above
cmd = """
      python run_language_modeling.py 
      --train_data_file ./train_data.txt 
      --eval_data_file ./eval_data.txt
      --output_dir ./TweetBERTa-v1 
      --model_type roberta 
      --model_name_or_path roberta-base
      --mlm 
      --config_name ./TweetBERTa 
      --tokenizer_name ./TweetBERTa 
      --do_train 
      --do_eval 
      --line_by_line 
      --learning_rate 1e-5 
      --num_train_epochs 5 
      --save_total_limit 20 
      --save_steps 5000 
      --per_gpu_train_batch_size 4
      --warmup_steps=10000 
      --logging_steps=100 
      --gradient_accumulation_steps=4 
      --seed 2020 --block_size=512
      """.replace("\n", " ")

In [0]:
%%time
#!{train_path}
#!{eval_path}
!{cmd}

Streaming output truncated to the last 5000 lines.
Iteration:  28% 23096/83459 [14:02<38:46, 25.94it/s]
Iteration:  28% 23100/83459 [14:02<37:25, 26.88it/s]
Iteration:  28% 23104/83459 [14:02<36:31, 27.54it/s]
Iteration:  28% 23108/83459 [14:02<35:59, 27.95it/s]
Iteration:  28% 23112/83459 [14:02<36:20, 27.67it/s]
Iteration:  28% 23116/83459 [14:02<36:15, 27.73it/s]
Iteration:  28% 23120/83459 [14:02<35:54, 28.01it/s]
Iteration:  28% 23124/83459 [14:03<35:51, 28.05it/s]
Iteration:  28% 23128/83459 [14:03<35:42, 28.16it/s]
Iteration:  28% 23132/83459 [14:03<35:35, 28.25it/s]
Iteration:  28% 23136/83459 [14:03<34:48, 28.89it/s]
Iteration:  28% 23140/83459 [14:03<35:17, 28.49it/s]
Iteration:  28% 23144/83459 [14:03<35:16, 28.50it/s]
Iteration:  28% 23148/83459 [14:03<34:43, 28.95it/s]
Iteration:  28% 23152/83459 [14:04<34:40, 28.98it/s]
Iteration:  28% 23156/83459 [14:04<34:24, 29.20it/s]
Iteration:  28% 23160/83459 [14:04<34:18, 29.30it/s]
Iteration:  28% 23164/83459 [14:04<35:24, 28.39i